In [9]:
import pandas as pd
import tensorflow as tf
import torch
import numpy as np
import time
import datetime
import random
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from nltk.stem.porter import *
from nltk.stem import PorterStemmer
from snowballstemmer import TurkishStemmer

import jpype
from typing import List
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java

import transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

In [10]:
train_df = pd.read_csv('reviews.csv')
train_df.head()

,RID,SID,text,polarity,category,target
0,1000,1000:0,Manzara sahane evet ama servis rezalet.,negative,SERVICE#GENERAL,servis
1,1000,1000:0,Manzara sahane evet ama servis rezalet.,positive,AMBIENCE#GENERAL,Manzara
2,1000,1000:1,"Soguk su isteyince, soguk yok, butun sulari di...",negative,FOOD#STYLE_OPTIONS,mezenin
3,1000,1000:1,"Soguk su isteyince, soguk yok, butun sulari di...",negative,SERVICE#GENERAL,garson
4,1000,1000:2,"Yemekler iyi hos, lezzetler iyi ama heyecan ve...",positive,FOOD#QUALITY,lezzetler


In [11]:
print(train_df['category'][0].rpartition('#')[0])
print(train_df['category'][0].rpartition('#')[1])
print(train_df['category'][0].rpartition('#')[2])

SERVICE
#
GENERAL


In [12]:
train_df['aspect_term'] = train_df['category'].apply(lambda x : x.rpartition('#')[0])
train_df['aspect_category'] = train_df['category'].apply(lambda x : x.rpartition('#')[2])

In [13]:
train_df.head()

,RID,SID,text,polarity,category,target,aspect_term,aspect_category
0,1000,1000:0,Manzara sahane evet ama servis rezalet.,negative,SERVICE#GENERAL,servis,SERVICE,GENERAL
1,1000,1000:0,Manzara sahane evet ama servis rezalet.,positive,AMBIENCE#GENERAL,Manzara,AMBIENCE,GENERAL
2,1000,1000:1,"Soguk su isteyince, soguk yok, butun sulari di...",negative,FOOD#STYLE_OPTIONS,mezenin,FOOD,STYLE_OPTIONS
3,1000,1000:1,"Soguk su isteyince, soguk yok, butun sulari di...",negative,SERVICE#GENERAL,garson,SERVICE,GENERAL
4,1000,1000:2,"Yemekler iyi hos, lezzetler iyi ama heyecan ve...",positive,FOOD#QUALITY,lezzetler,FOOD,QUALITY


In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1535 entries, 0 to 1534
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   RID              1535 non-null   int64 
 1   SID              1535 non-null   object
 2   text             1535 non-null   object
 3   polarity         1535 non-null   object
 4   category         1535 non-null   object
 5   target           1385 non-null   object
 6   aspect_term      1535 non-null   object
 7   aspect_category  1535 non-null   object
dtypes: int64(1), object(7)
memory usage: 96.1+ KB


#### CATEGORY COUNTS

In [15]:
train_df['category'].value_counts()

FOOD#QUALITY                446
AMBIENCE#GENERAL            277
SERVICE#GENERAL             235
RESTAURANT#GENERAL          228
FOOD#STYLE_OPTIONS          123
RESTAURANT#PRICES            65
DRINKS#QUALITY               51
LOCATION#GENERAL             39
FOOD#PRICES                  31
DRINKS#STYLE_OPTIONS         30
DRINKS#PRICES                 9
RESTAURANT#MISCELLANEOUS      1
Name: category, dtype: int64

#### ASPECT TERM COUNTS

In [16]:
train_df['aspect_term'].value_counts()

FOOD          600
RESTAURANT    294
AMBIENCE      277
SERVICE       235
DRINKS         90
LOCATION       39
Name: aspect_term, dtype: int64

#### ASPECT CATEGORY COUNTS

In [17]:
train_df['aspect_category'].value_counts()

GENERAL          779
QUALITY          497
STYLE_OPTIONS    153
PRICES           105
MISCELLANEOUS      1
Name: aspect_category, dtype: int64

### Preprocess

##### Data cleaning

-> Removing of punctuations, numbers, stopwords, urls etc.

-> Converting uppercase to lower

-> Tokenization

-> Lemmatization or Stemming

In [18]:
# create a set of stopwords
stopwords_set = stopwords.words('turkish')
for line in open('turkish_stopwords.txt','r').readlines():
    stopwords_set.append(line.strip())
    
print(stopwords_set)

['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani', 'a', 'acaba', 'altÄ±', 'altmÄ±ÅŸ', 'ama', 'ancak', 'arada', 'artÄ±k', 'asla', 'aslÄ±nda', 'aslÄ±nda', 'ayrÄ±ca', 'az', 'bana', 'bazen', 'bazÄ±', 'bazÄ±larÄ±', 'belki', 'ben', 'benden', 'beni', 'benim', 'beri', 'beÅŸ', 'bile', 'bilhassa', 'bin', 'bir', 'biraz', 'birÃ§oÄŸu', 'birÃ§ok', 'biri', 'birisi', 'birkaÃ§', 'birÅŸey', 'biz', 'bizden', 'bize', 'bizi', 'bizim', 'bÃ¶yle', 'bÃ¶ylece', 'bu', 'buna', 'bunda', 'bundan', 'bunlar', 'bunlarÄ±', 'bunlarÄ±n', 'bunu', 'bunun', 'burada', 'bÃ¼tÃ¼n', 'Ã§oÄŸu', 'Ã§oÄŸunu', 'Ã§ok', 'Ã§Ã¼nkÃ¼', 'da', 'daha', 'dahi', 'dan', 'de', 'defa', 'deÄŸil', 'diÄŸer'

In [19]:
# create a set of punctuations
punctuation_set = string.punctuation
punctuation_set

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [20]:
# Check has some unnecessary words. Find if exists and count
re_urls = r'http\S+'
re_html = r'<.*?>'
re_digits = r'\d+'

print("Number of urls in data ", train_df['text'].str.count(re_urls).sum())
print("Number of html tag in data ", train_df['text'].str.count(re_html).sum())
print("Number of digits in data ", train_df['text'].str.count(re_digits).sum())

Number of urls in data  0
Number of html tag in data  0
Number of digits in data  163


In [21]:
def clean(text):
    #lowercase operation
    text = text.lower()
    #remove digits
    text = re.sub(re_digits, '', text)
    #remove stop words 
    text = text.split()
    text = ' '.join([word for word in text if not word in stopwords_set])
    
    return text

In [22]:
def remove_punctuations(text):
    for punctuation in punctuation_set:
        text = text.replace(punctuation, ' ')
    return text

In [24]:
train_df['cleaned_text'] = train_df['text'].apply(lambda x: clean(x))

# CLEANING PUNCTUATIONS PROCESS
train_df["cleaned_text"] = train_df['cleaned_text'].apply(remove_punctuations)

## CLEANING STOPWORDS PROCESS
train_df['cleaned_text'] = train_df['cleaned_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_set)]))

train_df.head()

,RID,SID,text,polarity,category,target,aspect_term,aspect_category,cleaned_text
0,1000,1000:0,Manzara sahane evet ama servis rezalet.,negative,SERVICE#GENERAL,servis,SERVICE,GENERAL,manzara sahane evet servis rezalet
1,1000,1000:0,Manzara sahane evet ama servis rezalet.,positive,AMBIENCE#GENERAL,Manzara,AMBIENCE,GENERAL,manzara sahane evet servis rezalet
2,1000,1000:1,"Soguk su isteyince, soguk yok, butun sulari di...",negative,FOOD#STYLE_OPTIONS,mezenin,FOOD,STYLE_OPTIONS,soguk su isteyince soguk yok butun sulari disa...
3,1000,1000:1,"Soguk su isteyince, soguk yok, butun sulari di...",negative,SERVICE#GENERAL,garson,SERVICE,GENERAL,soguk su isteyince soguk yok butun sulari disa...
4,1000,1000:2,"Yemekler iyi hos, lezzetler iyi ama heyecan ve...",positive,FOOD#QUALITY,lezzetler,FOOD,QUALITY,yemekler iyi hos lezzetler iyi heyecan verici ...


In [26]:
### GPU CHECK
# check GPU
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    device = torch.device("cuda")
    print('GPU:', torch.cuda.get_device_name(0))
else:
    raise SystemError('GPU device not found')

AssertionError: Torch not compiled with CUDA enabled

### ASPECT PRED

### POLARITY PRED

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-128k-uncased', do_lower_case=True)
sentences = train_df.cleaned_text.values
max_len = 250

## Zemberek NLP


In [25]:
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)

morphology = TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)
extractor = TurkishSentenceExtractor()

2022-05-31 23:05:26,318 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 9.744978904724121



### Sentence Normalization

In [26]:
def normalize_long_text(text):
    normalized_sentences = [normalizer.normalize(word) for word in text]
    normalized_text = " ".join(normalized_sentences)
    return normalized_text

In [35]:
import time
import logging

logger = logging.getLogger(__name__)

sentences = train_df['cleaned_text'].copy()
new_sent = []
start = time.time()

for token in sentences:
    if token.count('') > 0:
        token = list(filter(('').__ne__, token))
    new_token = normalize_long_text(token)
    new_sent.append(new_token)

logger.info(f"Sentences normalized in: {time.time() - start} s")


NameError: name 'normalizer' is not defined

### Lemmazation

In [26]:
string_text = train_df['cleaned_text'].str
train_df['cleaned_text'] = string_text.translate(str.maketrans('', '', punctuation_set))
train_df['text_word'] = train_df['cleaned_text'].apply(word_tokenize).to_list()
train_df['text_word_count'] = train_df['text_word'].apply(len)

In [ ]:
train_df.to_csv('train_df.csv')

In [27]:
clean_sent = train_df['text_word']
clean_sent

0                [manzara, sahane, evet, servis, rezalet]
1                [manzara, sahane, evet, servis, rezalet]
2       [soguk, su, isteyince, soguk, yok, butun, sula...
3       [soguk, su, isteyince, soguk, yok, butun, sula...
4       [yemekler, iyi, hos, lezzetler, iyi, heyecan, ...
                              ...                        
1530      [tereyağlı, pilavları, tava, yoğurdu, başarılı]
1531    [üzerine, bal, fındık, dökülmüş, sütlaçları, c...
1532    [üzerine, bal, fındık, dökülmüş, sütlaçları, c...
1533    [ayrıca, hergun, çıkardıkları, farklı, zeytiny...
1534            [fiyatlar, ortalamanın, ustunde, malesef]
Name: text_word, Length: 1535, dtype: object

In [28]:
import zeyrek

analyzer = zeyrek.MorphAnalyzer()
lem_sent = []
for sent in clean_sent:
    normalized_sent = []
    for word in sent:
        if word == '':
            continue
        else:
            lem_word = analyzer.lemmatize(word)
            normalized_sent.append(lem_word[0][1][0])
    lem_sent.append(normalized_sent)

2022-05-31 23:05:49,277 - root - WARNING
Msg: APPENDING RESULT: <(manzara_Noun)(-)(manzara:noun_S + a3sg_S + pnon_S + nom_ST)>

2022-05-31 23:05:49,282 - root - WARNING
Msg: APPENDING RESULT: <(evet_Adv)(-)(evet:advRoot_ST)>

2022-05-31 23:05:49,282 - root - WARNING
Msg: APPENDING RESULT: <(evet_Interj)(-)(evet:interjRoot_ST)>

2022-05-31 23:05:49,286 - root - WARNING
Msg: APPENDING RESULT: <(servis_Noun)(-)(servis:noun_S + a3sg_S + pnon_S + nom_ST)>

2022-05-31 23:05:49,289 - root - WARNING
Msg: APPENDING RESULT: <(rezalet_Noun)(-)(rezalet:noun_S + a3sg_S + pnon_S + nom_ST)>

2022-05-31 23:05:49,291 - root - WARNING
Msg: APPENDING RESULT: <(manzara_Noun)(-)(manzara:noun_S + a3sg_S + pnon_S + nom_ST)>

2022-05-31 23:05:49,296 - root - WARNING
Msg: APPENDING RESULT: <(evet_Adv)(-)(evet:advRoot_ST)>

2022-05-31 23:05:49,297 - root - WARNING
Msg: APPENDING RESULT: <(evet_Interj)(-)(evet:interjRoot_ST)>

2022-05-31 23:05:49,301 - root - WARNING
Msg: APPENDING RESULT: <(servis_Noun)(-)(serv

In [29]:
x = lem_sent.copy()
for sent in x:
    i = 0
    for token in sent:
        sent[i] = token.lower()
        i += 1
lem_sent = x

In [30]:
lem_sent

[['manzara', 'sahane', 'evet', 'servis', 'rezalet'],
 ['manzara', 'sahane', 'evet', 'servis', 'rezalet'],
 ['soguk',
  'su',
  'istemek',
  'soguk',
  'yok',
  'butun',
  'sulari',
  'disari',
  'cikardik',
  'demek',
  'garson',
  'siparisten',
  'saat',
  'ge',
  'yemek',
  'yemegin',
  'yaninda',
  'soylenen',
  'roka',
  'pure',
  'eser',
  'olmamasi',
  'istedigimiz',
  'meze',
  'kalmamis',
  'olmasi'],
 ['soguk',
  'su',
  'istemek',
  'soguk',
  'yok',
  'butun',
  'sulari',
  'disari',
  'cikardik',
  'demek',
  'garson',
  'siparisten',
  'saat',
  'ge',
  'yemek',
  'yemegin',
  'yaninda',
  'soylenen',
  'roka',
  'pure',
  'eser',
  'olmamasi',
  'istedigimiz',
  'meze',
  'kalmamis',
  'olmasi'],
 ['yemek',
  'iyi',
  'hos',
  'lezzet',
  'iyi',
  'heyecan',
  'vermek',
  'bi',
  'taraflari',
  'yok',
  'iyi',
  'baligi',
  'iyi',
  'sekil',
  'izgara',
  'artik',
  'at',
  'dev',
  'bi',
  'olay',
  'degil'],
 ['yemek',
  'iyi',
  'hos',
  'lezzet',
  'iyi',
  'heyecan',

In [ ]:
def lemmatizer(sentence):
    # First, we need to tokenize sentences
    tokenized_sentence = word_tokenize(sentence)

    analyzer = zeyrek.MorphAnalyzer()
    lemmatized_sentence = []
    for word in tokenized_sentence:
        if word == '':
            continue
        else:
            lemmatized_word = analyzer.lemmatize(word)
            lemmatized_sentence.append(lemmatized_word[0][1][0])

    # Lowercase the tokens
    i = 0
    for word in lemmatized_sentence:
        lemmatized_sentence[i] = word.lower()
        i += 1

    return lemmatized_sentence

In [31]:
lem_sent = list(filter(('').__ne__, lem_sent))
lem_sent

[['manzara', 'sahane', 'evet', 'servis', 'rezalet'],
 ['manzara', 'sahane', 'evet', 'servis', 'rezalet'],
 ['soguk',
  'su',
  'istemek',
  'soguk',
  'yok',
  'butun',
  'sulari',
  'disari',
  'cikardik',
  'demek',
  'garson',
  'siparisten',
  'saat',
  'ge',
  'yemek',
  'yemegin',
  'yaninda',
  'soylenen',
  'roka',
  'pure',
  'eser',
  'olmamasi',
  'istedigimiz',
  'meze',
  'kalmamis',
  'olmasi'],
 ['soguk',
  'su',
  'istemek',
  'soguk',
  'yok',
  'butun',
  'sulari',
  'disari',
  'cikardik',
  'demek',
  'garson',
  'siparisten',
  'saat',
  'ge',
  'yemek',
  'yemegin',
  'yaninda',
  'soylenen',
  'roka',
  'pure',
  'eser',
  'olmamasi',
  'istedigimiz',
  'meze',
  'kalmamis',
  'olmasi'],
 ['yemek',
  'iyi',
  'hos',
  'lezzet',
  'iyi',
  'heyecan',
  'vermek',
  'bi',
  'taraflari',
  'yok',
  'iyi',
  'baligi',
  'iyi',
  'sekil',
  'izgara',
  'artik',
  'at',
  'dev',
  'bi',
  'olay',
  'degil'],
 ['yemek',
  'iyi',
  'hos',
  'lezzet',
  'iyi',
  'heyecan',

In [32]:
train_df['lemmatized'] = lem_sent
train_df['lemmatized'] = train_df['lemmatized'].apply(' '.join)

In [33]:
train_df.head()

,RID,SID,text,polarity,category,target,aspect_term,aspect_category,cleaned_text,text_word,text_word_count,lemmatized
0,1000,1000:0,Manzara sahane evet ama servis rezalet.,negative,SERVICE#GENERAL,servis,SERVICE,GENERAL,manzara sahane evet servis rezalet,"[manzara, sahane, evet, servis, rezalet]",5,manzara sahane evet servis rezalet
1,1000,1000:0,Manzara sahane evet ama servis rezalet.,positive,AMBIENCE#GENERAL,Manzara,AMBIENCE,GENERAL,manzara sahane evet servis rezalet,"[manzara, sahane, evet, servis, rezalet]",5,manzara sahane evet servis rezalet
2,1000,1000:1,"Soguk su isteyince, soguk yok, butun sulari di...",negative,FOOD#STYLE_OPTIONS,mezenin,FOOD,STYLE_OPTIONS,soguk su isteyince soguk yok butun sulari disa...,"[soguk, su, isteyince, soguk, yok, butun, sula...",26,soguk su istemek soguk yok butun sulari disari...
3,1000,1000:1,"Soguk su isteyince, soguk yok, butun sulari di...",negative,SERVICE#GENERAL,garson,SERVICE,GENERAL,soguk su isteyince soguk yok butun sulari disa...,"[soguk, su, isteyince, soguk, yok, butun, sula...",26,soguk su istemek soguk yok butun sulari disari...
4,1000,1000:2,"Yemekler iyi hos, lezzetler iyi ama heyecan ve...",positive,FOOD#QUALITY,lezzetler,FOOD,QUALITY,yemekler iyi hos lezzetler iyi heyecan verici ...,"[yemekler, iyi, hos, lezzetler, iyi, heyecan, ...",21,yemek iyi hos lezzet iyi heyecan vermek bi tar...


In [35]:
train_df.to_csv('train_df_2.csv', index=False)

In [39]:
train_df_2 = pd.read_csv('train_df_2.csv')
lemmatized2 = train_df_2['lemmatized']

In [46]:
lemmatized1

0                      manzara sahane evet servis rezalet
1                      manzara sahane evet servis rezalet
2       soguk su istemek soguk yok butun sulari disari...
3       soguk su istemek soguk yok butun sulari disari...
4       yemek iyi hos lezzet iyi heyecan veri bi taraf...
                              ...                        
1530                      tereyağ pilâv tav yoğurt başarı
1531    üzerine bal fındık dökmek sütlâç cezbetmek göz...
1532    üzerine bal fındık dökmek sütlâç cezbetmek göz...
1533     ayrı hergun çıkarmak fark zeytinyağ yemek başarı
1534                       fiyat ortalama ustunde malesef
Name: lemmatized, Length: 1535, dtype: object

In [49]:
freq = pd.Series(' '.join(lemmatized2).split()).value_counts()
less_freq = list(freq[freq == 1].index)

TypeError: sequence item 164: expected str instance, float found

In [35]:
train_df['lemmatized'] = train_df['lemmatized'].apply(lambda x: " ".join(x for x in x.split() if x not in less_freq))
train_df['lemmatized'].head(5)

0                   manzara sahane evet servis rezalet
1                   manzara sahane evet servis rezalet
2    soguk su istemek soguk yok butun sulari disari...
3    soguk su istemek soguk yok butun sulari disari...
4    yemek iyi hos lezzet iyi heyecan vermek bi tar...
Name: lemmatized, dtype: object

In [47]:
train_df.head()

,RID,SID,text,polarity,category,target,aspect_term,aspect_category,cleaned_text,text_word,text_word_count,lemmatized
0,1000,1000:0,Manzara sahane evet ama servis rezalet.,negative,SERVICE#GENERAL,servis,SERVICE,GENERAL,manzara sahane evet servis rezalet,"[manzara, sahane, evet, servis, rezalet]",5,manzara sahane evet servis rezalet
1,1000,1000:0,Manzara sahane evet ama servis rezalet.,positive,AMBIENCE#GENERAL,Manzara,AMBIENCE,GENERAL,manzara sahane evet servis rezalet,"[manzara, sahane, evet, servis, rezalet]",5,manzara sahane evet servis rezalet
2,1000,1000:1,"Soguk su isteyince, soguk yok, butun sulari di...",negative,FOOD#STYLE_OPTIONS,mezenin,FOOD,STYLE_OPTIONS,soguk su isteyince soguk yok butun sulari disa...,"[soguk, su, isteyince, soguk, yok, butun, sula...",26,soguk su istemek soguk yok butun sulari disari...
3,1000,1000:1,"Soguk su isteyince, soguk yok, butun sulari di...",negative,SERVICE#GENERAL,garson,SERVICE,GENERAL,soguk su isteyince soguk yok butun sulari disa...,"[soguk, su, isteyince, soguk, yok, butun, sula...",26,soguk su istemek soguk yok butun sulari disari...
4,1000,1000:2,"Yemekler iyi hos, lezzetler iyi ama heyecan ve...",positive,FOOD#QUALITY,lezzetler,FOOD,QUALITY,yemekler iyi hos lezzetler iyi heyecan verici ...,"[yemekler, iyi, hos, lezzetler, iyi, heyecan, ...",21,yemek iyi hos lezzet iyi heyecan vermek bi tar...


In [ ]:
type(train_df.head(10))

In [93]:
type(train_df['text'])

0                      manzara sahane evet servis rezalet
1                      manzara sahane evet servis rezalet
2       soguk su istemek soguk yok butun sulari disari...
3       soguk su istemek soguk yok butun sulari disari...
4       yemek iyi hos lezzet iyi heyecan vermek bi tar...
                              ...                        
1530                    tereyağ pilâv tav yoğurmak başarı
1531    üzer bal fındık dökmek sütlâç cezbetmek gözükm...
1532    üzer bal fındık dökmek sütlâç cezbetmek gözükm...
1533                   ayrıca fark zeytinyağ yemek başarı
1534                         fiyatlamak ortalamak malesef
Name: lemmatized2, Length: 1535, dtype: object

In [51]:
train_df

,RID,SID,text,polarity,category,target,aspect_term,aspect_category,cleaned_text,text_word,text_word_count,lemmatized,lemmatized2
0,1000,1000:0,Manzara sahane evet ama servis rezalet.,negative,SERVICE#GENERAL,servis,SERVICE,GENERAL,manzara sahane evet servis rezalet,"[manzara, sahane, evet, servis, rezalet]",5,manzara sahane evet servis rezalet,manzara sahane evet servis rezalet
1,1000,1000:0,Manzara sahane evet ama servis rezalet.,positive,AMBIENCE#GENERAL,Manzara,AMBIENCE,GENERAL,manzara sahane evet servis rezalet,"[manzara, sahane, evet, servis, rezalet]",5,manzara sahane evet servis rezalet,manzara sahane evet servis rezalet
2,1000,1000:1,"Soguk su isteyince, soguk yok, butun sulari di...",negative,FOOD#STYLE_OPTIONS,mezenin,FOOD,STYLE_OPTIONS,soguk su isteyince soguk yok butun sulari disa...,"[soguk, su, isteyince, soguk, yok, butun, sula...",26,soguk su istemek soguk yok butun sulari disari...,soguk su istemek soguk yok butun sulari disari...
3,1000,1000:1,"Soguk su isteyince, soguk yok, butun sulari di...",negative,SERVICE#GENERAL,garson,SERVICE,GENERAL,soguk su isteyince soguk yok butun sulari disa...,"[soguk, su, isteyince, soguk, yok, butun, sula...",26,soguk su istemek soguk yok butun sulari disari...,soguk su istemek soguk yok butun sulari disari...
4,1000,1000:2,"Yemekler iyi hos, lezzetler iyi ama heyecan ve...",positive,FOOD#QUALITY,lezzetler,FOOD,QUALITY,yemekler iyi hos lezzetler iyi heyecan verici ...,"[yemekler, iyi, hos, lezzetler, iyi, heyecan, ...",21,yemek iyi hos lezzet iyi heyecan vermek bi tar...,yemek iyi hos lezzet iyi heyecan vermek bi tar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,1299,1299:1,Tereyağlı pilavları ve tava yoğurdu da çok baş...,positive,FOOD#QUALITY,Tereyağlı pilavları,FOOD,QUALITY,tereyağlı pilavları tava yoğurdu başarılı,"[tereyağlı, pilavları, tava, yoğurdu, başarılı]",5,tereyağ pilâv tav yoğurmak başarı,tereyağ pilâv tav yoğurmak başarı
1531,1299,1299:2,Üzerine bal ve fındık dökülmüş sütlaçları çok ...,negative,FOOD#QUALITY,sütlaç tadını,FOOD,QUALITY,üzerine bal fındık dökülmüş sütlaçları cezbedi...,"[üzerine, bal, fındık, dökülmüş, sütlaçları, c...",12,üzer bal fındık dökmek sütlâç cezbetmek gözükm...,üzer bal fındık dökmek sütlâç cezbetmek gözükm...
1532,1299,1299:2,Üzerine bal ve fındık dökülmüş sütlaçları çok ...,positive,FOOD#STYLE_OPTIONS,sütlaçları,FOOD,STYLE_OPTIONS,üzerine bal fındık dökülmüş sütlaçları cezbedi...,"[üzerine, bal, fındık, dökülmüş, sütlaçları, c...",12,üzer bal fındık dökmek sütlâç cezbetmek gözükm...,üzer bal fındık dökmek sütlâç cezbetmek gözükm...
1533,1299,1299:3,Ayrıca hergun çıkardıkları farklı zeytinyağlı ...,positive,FOOD#QUALITY,zeytinyağlı yemekleri,FOOD,QUALITY,ayrıca hergun çıkardıkları farklı zeytinyağlı ...,"[ayrıca, hergun, çıkardıkları, farklı, zeytiny...",7,ayrıca fark zeytinyağ yemek başarı,ayrıca fark zeytinyağ yemek başarı


## Data visualization

In [ ]:
from powerbiclient import Report, models

# Import the DeviceCodeLoginAuthentication class to authenticate against Power BI
from powerbiclient.authentication import DeviceCodeLoginAuthentication

# Initiate device authentication
device_auth = DeviceCodeLoginAuthentication()

In [ ]:
report_id="8c128070-7ec4-4441-8b6d-dfbe1b5dd8a1"

report = Report(report_id=report_id, group_id=None, auth=device_auth)

report

In [ ]:
report = Report(group_id=group_id, report_id=report_id, access_token=access_token, token_type=models.TokenType.Embed.value)

![](visualizations.png)